# Math 271.1: Exercise 2 (#6)

#### INSTRUCTION: Tikhonov Regularization for an Ill-posed Example <br>

(a) Compute the direct (unregularized) solutions <br>
(b) the Tikhonov-regularized solutions with fixed parameter λ= 10−3 <br>
(c) For both the direct and regularized cases, compute and report